# --- Day 7: Recursive Circus ---

Wandering further through the circuits of the computer, you come upon a tower of programs that have gotten themselves into a bit of trouble. A recursive algorithm has gotten out of hand, and now they're balanced precariously in a large tower.

One program at the bottom supports the entire tower. It's holding a large disc, and on the disc are balanced several more sub-towers. At the bottom of these sub-towers, standing on the bottom disc, are other programs, each holding their own disc, and so on. At the very tops of these sub-sub-sub-...-towers, many programs stand simply keeping the disc below them balanced but with no disc of their own.

You offer to help, but first you need to understand the structure of these towers. You ask each program to yell out their name, their weight, and (if they're holding a disc) the names of the programs immediately above them balancing on that disc. You write this information down (your puzzle input). Unfortunately, in their panic, they don't do this in an orderly fashion; by the time you're done, you're not sure which program gave which information.

For example, if your list is the following:

```
pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)
```

...then you would be able to recreate the structure of the towers that looks like this:

```
                gyxo
              /     
         ugml - ebii
       /      \     
      |         jptl
      |        
      |         pbga
     /        /
tknk --- padx - havc
     \        \
      |         qoyq
      |             
      |         ktlj
       \      /     
         fwft - cntj
              \     
                xhth
```

In this example, tknk is at the bottom of the tower (the bottom program), and is holding up ugml, padx, and fwft. Those programs are, in turn, holding up other programs; in this example, none of those programs are holding up any other programs, and are all the tops of their own towers. (The actual tower balancing in front of you is much larger.)

Before you're ready to help them, you need to make sure your information is correct. What is the name of the bottom program?

## a non graph hacky way to answer this

The obvious way to answer this is to build a graph, but since the problem is so well defined - there is only one parent node, we can figure this out without lists.

First, the test input:

In [37]:
test_case = """pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)""".split("\n")

print(f"the test case answer is: tknk")
test_case

the test case answer is: tknk


['pbga (66)',
 'xhth (57)',
 'ebii (61)',
 'havc (66)',
 'ktlj (57)',
 'fwft (72) -> ktlj, cntj, xhth',
 'qoyq (66)',
 'padx (45) -> pbga, havc, qoyq',
 'tknk (41) -> ugml, padx, fwft',
 'jptl (61)',
 'ugml (68) -> gyxo, ebii, jptl',
 'gyxo (61)',
 'cntj (57)']

Now a list of all the nodes:

In [3]:
nodes = [line.split()[0] for line in test_case]
nodes

['pbga',
 'xhth',
 'ebii',
 'havc',
 'ktlj',
 'fwft',
 'qoyq',
 'padx',
 'tknk',
 'jptl',
 'ugml',
 'gyxo',
 'cntj']

Now a list of just the parent nodes (the ones which have children):

In [4]:
parents = [line.split()[0] for line in test_case if "->" in line]
parents

['fwft', 'padx', 'tknk', 'ugml']

In [5]:
children_nodes = [line.split()[3:] for line in test_case if "->" in line]
children_nodes

[['ktlj,', 'cntj,', 'xhth'],
 ['pbga,', 'havc,', 'qoyq'],
 ['ugml,', 'padx,', 'fwft'],
 ['gyxo,', 'ebii,', 'jptl']]

I want a nice flat list of all the children nodes, so I use chain to flatten out the list, and also here I strip the punctuation characters from the nodes:

In [6]:
import re
from itertools import chain
childs = [re.sub('[\W_]+', "", node) for node in chain(*children_nodes)]
childs

['ktlj',
 'cntj',
 'xhth',
 'pbga',
 'havc',
 'qoyq',
 'ugml',
 'padx',
 'fwft',
 'gyxo',
 'ebii',
 'jptl']

the answer should just be:

In [7]:
[node for node in parents if node not in childs]

['tknk']

Now to make it into a function:

In [8]:
from itertools import chain
    
def top_node(test_case):
    """takes in a tree in text form and returns the name of the parent node"""
    parents = [line.split()[0] for line in test_case if "->" in line]
    children_nodes = [line.split()[3:] for line in test_case if "->" in line]
    childs = [re.sub('[\W_]+', "", node) for node in chain(*children_nodes)]
    return [node for node in parents if node not in childs]

top_node(test_case)

['tknk']

Now to read in the puzzle data and get the parent node:

In [213]:
with open('puzzle_inputs/day7_input.txt') as f:
    data = f.read().split("\n")
data = [line for line in data if len(line)>0]
data[:8]

['tqefb (40)',
 'lhrml (164) -> ecblhee, sdjshz',
 'ykntwjm (16)',
 'fbebcq (233) -> ilzfg, vqbvnf, idyiyg, tifpswp',
 'rqjpza (1043) -> xszbzi, zafhcbb, qoouyiw',
 'zazkyd (203) -> vzylhou, ndvkjn',
 'ndfxn (48) -> brxmlaa, nlbvsaj',
 'pfjpsxf (1714) -> uchxwm, ohpvb']

In [214]:
top_node(data)

['cyrupz']

And that works! Though my solution only deals with this puzzles case, a better one would be to buid a gragh, which it turns out I need to do for part 2 anyways...

# --- Part Two ---

The programs explain the situation: they can't get down. Rather, they could get down, if they weren't expending all of their energy trying to keep the tower balanced. Apparently, one program has the wrong weight, and until it's fixed, they're stuck here.

For any program holding a disc, each program standing on that disc forms a sub-tower. Each of those sub-towers are supposed to be the same weight, or the disc itself isn't balanced. The weight of a tower is the sum of the weights of the programs in that tower.

In the example above, this means that for ugml's disc to be balanced, gyxo, ebii, and jptl must all have the same weight, and they do: 61.

However, for tknk to be balanced, each of the programs standing on its disc and all programs above it must each match. This means that the following sums must all be the same:

```
ugml + (gyxo + ebii + jptl) = 68 + (61 + 61 + 61) = 251
padx + (pbga + havc + qoyq) = 45 + (66 + 66 + 66) = 243
fwft + (ktlj + cntj + xhth) = 72 + (57 + 57 + 57) = 243
```

As you can see, tknk's disc is unbalanced: ugml's stack is heavier than the other two. Even though the nodes above ugml are balanced, ugml itself is too heavy: it needs to be 8 units lighter for its stack to weigh 243 and keep the towers balanced. If this change were made, its weight would be 60.

Given that exactly one program is the wrong weight, what would its weight need to be to balance the entire tower?

---

I'm going to build a tower, and since we already know the bottom node from above, I'll start with that one. First, a class to hold the tower and all its sub towers:

In [215]:
class Tower:
    def __init__(self, name, weight, parent=None, sub_towers=[]):
        self.name = name
        self.weight = weight
        self.parent = parent
        self.sub_towers = sub_towers
    
    def total_weight(self):
        """returns total weight of the tower"""
        if self.sub_towers!=[]:
            sub_towers_weight = sum([t.total_weight() for t in self.sub_towers])
            return self.weight + sub_towers_weight
        else:
            return self.weight
        
    def __str__(self):
        subs = [tower.name for tower in self.sub_towers if self.sub_towers!=[]]
        return f"{self.name}, {self.weight}, has {subs} subtowers"

Now a helper function to parse each nodes description:

In [218]:
def parse_line(tower_desc):
    """takes in a line describing a tower and returns its name, weight, sub_towers"""
    tower = tower_desc.split()
    name = tower[0]
    weight = re.sub('[\W_]+', "", tower[1])
    sub_towers = []
    # deal with subtowers
    if len(tower) > 3:
        sub_towers_names = [re.sub('[\W]+', "", node) for node in tower[3:]]
        sub_towers = [st for st in sub_towers_names]
        
    return name, int(weight), sub_towers

parse_line('fbebcq (233) -> ilzfg, vqbvnf, idyiyg, tifpswp')

('fbebcq', 233, ['ilzfg', 'vqbvnf', 'idyiyg', 'tifpswp'])

Now I'm going to use the Tower class and the parse_line function to recursively build out the entire tower:

In [219]:
first_node = top_node(test_case)[0]
print(f"the first node is {first_node}")

def build_tower(node, tower_description, parent=None):
    """takes in a node name and list of strings 
    representing the nodes, weights and their sub nodes, returns a tower"""
    all_nodes = [line.split()[0] for line in tower_description]
    node_index = all_nodes.index(node)
    node_desc = tower_description[node_index]
    
    name, weight, sub_towers = parse_line(node_desc)
    
    tower = Tower(name, weight, parent, sub_towers)
    if sub_towers:
        for i, sub_t in enumerate(sub_towers):
            sub_towers[i] = build_tower(sub_t, tower_description, name)       
    return tower
    
tower = build_tower(first_node, test_case)
print(tower)
print(f"the towers weight: {tower.total_weight()}")

the first node is tknk
tknk, 41, has ['ugml', 'padx', 'fwft'] subtowers
the towers weight: 778


Since only one tower is imbalanced, I'll go down through the tower, looking at its subtowers. Each subtower should have the same total weight, if it doesn't then that tower is causing the imbalance.

In [253]:
from collections import Counter

def find_unbalanced_subtower(tower):
    """takes in a tower and returns the unbalanced sub_tower"""
    
    if not tower.sub_towers:
        return tower.weight
    
    unbalanced_tower, balanced_weight = None, 0
    
    names = [s.name for s in tower.sub_towers]
    weights = [s.total_weight() for s in tower.sub_towers]
    
    # check if this is the unbalanced tower
    if len(set(weights)) > 1:
        for weight, count in Counter(weights).items():
            if count == 1:
                unbalanced_weight = weight
            else:
                balanced_weight = weight
        print("found unbalance tower")
        unbalanced_tower = tower.sub_towers[weights.index(unbalanced_weight)]
        
        for t in unbalanced_tower.sub_towers:
            find_unbalanced_subtower(t)
            
        return unbalanced_tower, balanced_weight
    else:
        for t in tower.sub_towers:
            find_unbalanced_subtower(t)
    
    return None
        
ans, balanced_weight = find_unbalanced_subtower(tower)
print(ans)
# weight shold be:
print(ans.name, "weight should be:", ans.weight - (ans.total_weight() - balanced_weight))

found unbalance tower
ugml, 68, has ['gyxo', 'ebii', 'jptl'] subtowers
ugml weight should be: 60


Now to solve the test input:

In [254]:
first_node = top_node(data)[0]
print(f"the first node is {first_node}")
tower2 = build_tower(first_node, data)

ans, balanced_weight = find_unbalanced_subtower(tower2)
print(ans)
print(ans.weight, ans.total_weight(), balanced_weight)
print(ans.name, "weight should be:", ans.weight - (ans.total_weight() - balanced_weight))

the first node is cyrupz
found unbalance tower
found unbalance tower
qjvtm, 82986, has ['myfhxk', 'boropxd', 'jixdvf'] subtowers
82986 119432 119424
qjvtm weight should be: 82978


So we need to go down this node and find the unbalanced node:

In [248]:
ans2, balanced_weight = find_unbalanced_subtower(ans)
print(ans2.weight, ans2.total_weight(), balanced_weight)
print(ans2.name, "weight should be:", ans2.weight - (ans2.total_weight() - balanced_weight))

found unbalance tower
4285 12154 12146
boropxd weight should be: 4277


In [256]:
ans3, balanced_weight = find_unbalanced_subtower(ans2)
print(ans3.weight, ans3.total_weight(), balanced_weight)
print(ans3.name, "weight should be:", ans3.weight - (ans3.total_weight() - balanced_weight))

found unbalance tower
201 1131 1123
cwwwj weight should be: 193
